# Valence value regression based on Deap Dataset
## 1. FFT with pyeeg
* [4-8]: theta band
* [8-12]: alpha band
* [12-16]: low beta band 
* [16-25]: high beta band
* [25-45]: gamma band

In [1]:
import numpy as np
import pyeeg as pe
import pickle as pickle
import pandas as pd

## 2. Channel Indice 
File Name data\SXX.dat, XX \in [0,31]
* data: 40 x 40 x 8064: trial x channel x data
* label: 40 x 4: video/trial x label (valence, arousal, dominance, liking)

Channel Indice: {
* 1 : AF3
* 2: F3
* 3: F7
* 4: FC5
* 7: T7
* 11: P7
* 13: O1
* 17: AF4
* 19: F4
* 20: F8
* 21: FC6
* 25: T8
* 29: P8
* 31: O2 }



In [62]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #averaging band power of 2 sec
step_size = 16 #each 0.125 sec update once
sample_rate = 128 #sampling rate of 128 Hz

def FFT_Processing (sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indice
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    
    meta = []
    with open('data\s' + sub + '.dat', 'rb') as file:

        subject = pickle.load(file, encoding='latin1') #resolve the python 2 data problem by encoding : latin1

        for i in range (0,40):
            # loop over 0-39 trails
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0;

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = [] #meta vector for analysis
                for j in channel:
                    X = data[j][start : start + window_size] #Slice raw data over 2 sec, at interval of 0.125 sec
                    Y = pe.bin_power(X, band, sample_rate) #FFT over 2 sec of channel j, in seq of theta, alpha, low beta, high beta, gamma
                    meta_data = meta_data + list(Y[0])

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start = start + step_size
        meta = np.array(meta)
        np.save('out\s' + sub, meta, allow_pickle=True, fix_imports=True)


subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']

for subjects in subjectList:
    FFT_Processing (subjects, channel, band, window_size, step_size, sample_rate)

## 3.Segment and training of preprocessed data
* training dataset
* testing dataset
* validation dataset

Agrithom pool:
* Support Vector Machine (which kernal?)
* Ada-Boost

Best practice could be refered to this paper: 

@article{alarcao2017emotions,
  title={Emotions recognition using EEG signals: A survey},
  author={Alarcao, Soraia M and Fonseca, Manuel J},
  journal={IEEE Transactions on Affective Computing},
  year={2017},
  publisher={IEEE}
}